# Recurrent Neural Net From Scratch

I will skip the reasoning behind why we need RNN. There's plenty on the internet. I'll focus on learning the technical side, and hope that the answers to "why"s will come naturally. This is the same principal as other notebooks in my "from scratch" series. This is heavily based on the basic neural net, so read that one first.

Recall that in a neural network, we have:

![nn](imgs/nn.png)

For recurrent neural network, we carry over info from the previous data point in the sequence, by incorporating the values of the hidden layer, with its own weight $u$.

![nn](imgs/recurrent_neuralnet.png)

Rephrasing the diagram, we have the feed forward process as
 * $s_h = xw + a_h^{prev}u + b_h$
 * $a_h = \sigma_h(s_h)$
 * $s_o = a_hv +b_o$
 * $y = \sigma_o(s_o)$

h means hidden layer, o means output layer, s means weighted sum, a means activated value. $\sigma$ means the activation function. For the activation functions, we use softmax at the output layer and tanh at the hidden layer.

$$\large
tanh(x) = \frac{e^x - e^{-x}}{e^x + e^{-x}}
$$

## Backpropagation
* Backpropagation from the output layer to the hidden layer remains the same since we don't add anything new between these two layers
* Backpropagation from the hidden layer will go to both the input layer and the hidden layer of the previous data point.